In [43]:
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob

# from os import system as term
# from subprocess import Popen as term
from subprocess import call as term


In [27]:
script_dir = '/uufs/chpc.utah.edu/common/home/u1070830/code/snow-liquid-ratio/'
obs_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'
era5_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/'
gfs_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/gfs/'

## Start with the metadata file as an efficent way to keep data organized vs a config file

In [39]:
metadata = pd.read_excel(obs_path + 'Dataset_Metadata.xlsx').set_index('code')
metadata.iloc[:, :-1]

,name,owner,city,state,lat,lon,elevation_m,start,end,interval,n_events,auto_manual,snow_type,swe_type
code,,,,,,,,,,,,,,
CLN,Alta Collins,Alta Ski Lifts,Alta,UT,40.576300,-111.638300,2945,NaN,NaN,12/24,NaN,Manual,Snow Board (HSN),NaN
AGD,Alta Guard,Utah Dept. of Transportation,Alta,UT,40.590500,-111.638000,2682,NaN,NaN,6/12/24,NaN,Manual,Snow Board (HSN),NaN
ALTA,Alta CoOp,National Weather Service,Alta,UT,40.590580,-111.637030,2655,NaN,NaN,12/24,NaN,Manual,NaN,NaN
BCC,Big Cottonwood,Utah Dept. of Transportation,Brighton,UT,40.650300,-111.649900,2224,NaN,NaN,12/24,NaN,Manual,NaN,NaN
PVC,Provo Canyon,Utah Dept. of Transportation,Provo,UT,40.334100,-111.613300,1560,NaN,NaN,12/24,NaN,Manual,NaN,NaN
SLB,Silver Lake Brighton,National Weather Service,Brighton,UT,40.600800,-111.584200,2664,NaN,NaN,24,NaN,Manual,NaN,NaN
BSNFDC,Devil's Creek,Burlington Northern Santa Fe Corp,NaN,MT,48.277725,-113.432578,1415,NaN,NaN,NaN,NaN,Manual,NaN,NaN
BSNFEX,Essex,Burlington Northern Santa Fe Corp,Essex,MT,48.281760,-113.606570,1173,NaN,NaN,NaN,NaN,Manual,NaN,NaN
BSNFJE,Java East,Burlington Northern Santa Fe Corp,NaN,MT,48.235870,-113.564530,1232,NaN,NaN,NaN,NaN,Manual,NaN,NaN


## Loop over each station:
We don't want to re-run on data already processed, so check each step first

- Ensure the observation data exists and is read in to .pd

In [47]:
site_list = metadata.index.values.astype('str')
print('Sites to process:', site_list)

ingest_script = script_dir + 'data-tools/generic_data_ingest.py'

for site in site_list:
    
    try:
        run_cmd = ('python ' + ingest_script + ' %s'%site)
        term(run_cmd, shell=True)
        
    except:
        raise
        #print('Ingest %s failed'%site)
        
    else:
        # Check to see if file was written before declaring success - 
        # call won't catch failures in the child script
        print('Ingest %s success'%site)
    
# Pass cases:
#     Produced new .pd archive
#     Existing .pd archive found

# Fail cases:
#     No header info for site/file
#     No file found for site
#     Header mismatch/read error
#     Data mismatch error (automated - manual too large)

Sites to process: ['CLN' 'AGD' 'ALTA' 'BCC' 'PVC' 'SLB' 'BSNFDC' 'BSNFEX' 'BSNFJE']
Ingest CLN success
Ingest AGD success
Ingest ALTA success
Ingest BCC success
Ingest PVC success
Ingest SLB success
Ingest BSNFDC success
Ingest BSNFEX success
Ingest BSNFJE success


- Ensure the era5 profile has been created

In [4]:
# Pass cases:
#     Produced new ERA5 profile
#     ERA5 profile found

- Pair the observations and ERA5 profile

In [ ]:
# Pass cases:
#     Produced new ERA5 profile
#     ERA5 profile found

- Extract a GFS profile for verification

In [ ]:
# Pass cases:
#     Produced new GFS profile
#     GFS profile found

In [6]:
print('Data Pre-Processing Completed...')

Data Pre-Processing Completed...
